# Readme

Pre-requisites
*   Set up weight and bias account: [Wandb](https://wandb.ai/site/)
*   Upload dataset on drive following the hierarchy below: <br>
<pre>
MyDrive
|_ Final_project
   |_ 2000dataset.zip
</pre>

*   Upload the zip containing the code in drive following the heirarchy below:
<pre>
MyDrive
|_ Final_project
   |_ SwinIR
      |_ SwinIR.zip
</pre>

*   for **inference** using generator obtained after fine tunig on the dataset, ensure 5000_G is at the hierarchy below:

 <pre>
MyDrive
|_ Final_project
   |_ SwinIR
      |_ 5000_G.pth
</pre>


**Fine-Tuning**: Follow steps 1 to 4 in order. <br> <br>
**Inference**:
*   The SwinIR code from Kair repository and the generator is required for inference. Generator can be the one obtained after fine tuning with _G.pth or the original pretrained weight avilable at [JingyunLiang/SwinIR.git](http://github.com/JingyunLiang/SwinIR/releases/tag/v0.0)
*   Run step 1 and Step 5. *Note: At step 5.2, replace the image path with correct image path or folder path.*

# 1 Install dependencies and upload SwinIR code

In [ ]:
!pip install basicsr
!pip install wandb -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 14.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.0/303.0 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 22.6 MB/s eta 0:00:00
  Created wheel for basicsr: filename=basicsr-1.4.2-py3-none-any.whl size=214817 sha256=35b3e3a4cd0129987966af250445382e8ff5211230e53f7f439b9fcecfdb7f25
  Stored in directory: /root/.cache/pip/wheels/9a/e3/e4/58f29bfabb622dd40b6d9839318ce5bf092062b81ca3aa19ea
Successfully built basicsr


In [ ]:
#basicssr contains a file degradation.py where the import library is obsolete
#and has to be replaced
#https://github.com/xinntao/Real-ESRGAN/issues/801

file_path = "/usr/local/lib/python3.12/dist-packages/basicsr/data/degradations.py"
line_to_replace = "from torchvision.transforms.functional_tensor import rgb_to_grayscale"
new_line = "from torchvision.transforms.functional import rgb_to_grayscale"

with open(file_path, 'r') as file:
    lines = file.readlines()

lines = [new_line if line.strip() == line_to_replace else line for line in lines]

with open(file_path, 'w') as file:
    file.writelines(lines)

print(f"Replacing line '{line_to_replace}' with '{new_line}' in file {file_path}")

Replacing line 'from torchvision.transforms.functional_tensor import rgb_to_grayscale' with 'from torchvision.transforms.functional import rgb_to_grayscale' in file /usr/local/lib/python3.12/dist-packages/basicsr/data/degradations.py


In [ ]:
from google.colab import files, drive
import zipfile
import os
import torch
import json
import shutil
import wandb
import basicsr

In [ ]:
# log in to Weights & Biases; not needed of doing inference only
#
wandb.login()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dsha43925 (dsha43925-middlesex-university-mauritius) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
# get code from drive
drive.mount('/content/drive', force_remount=True)
code_zip_path = '/content/drive/MyDrive/Final_project/SwinIR/SwinIR.zip'
unzip_path = '/content'
!unzip -q $code_zip_path -d $unzip_path
!ls -l $unzip_path

Mounted at /content/drive
total 12
drwx------ 5 root root 4096 Sep 14 17:09 drive
drwxrwxrwx 8 root root 4096 Sep 13 17:33 KAIR
drwxr-xr-x 1 root root 4096 Sep  9 13:46 sample_data


In [ ]:
%cd /content/KAIR
!pip install sewar
!pip install -r /content/KAIR/requirement.txt

/content/KAIR
  Preparing metadata (setup.py) ... done
  Created wheel for sewar: filename=sewar-0.4.6-py3-none-any.whl size=11418 sha256=5d14d2cd9ccd08e61232e3bd878098cc3169dd6eace4d39681e4858f11722892
  Stored in directory: /root/.cache/pip/wheels/fb/4e/29/b15a3d425c5f0fe8f461cbfdaf4fa98ef203fed97ce1df6695
Successfully built sewar
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 18.3 MB/s eta 0:00:00


# 2 Get pre-trained models

following authors guide for fine tuning at https://github.com/cszn/KAIR/issues/94

In [ ]:
!wget https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x4_GAN-with-dict-keys-params-and-params_ema.pth -O /content/KAIR/model_zoo/003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x4_GAN-with-dict-keys-params-and-params_ema.pth

--2025-09-14 17:10:05--  https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x4_GAN-with-dict-keys-params-and-params_ema.pth
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/396770997/ad377089-20a9-4d77-87fd-1d85be80e1b4?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-09-14T18%3A05%3A03Z&rscd=attachment%3B+filename%3D003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x4_GAN-with-dict-keys-params-and-params_ema.pth&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-09-14T17%3A04%3A56Z&ske=2025-09-14T18%3A05%3A03Z&sks=b&skv=2018-11-09&sig=MN504sTC3Lv8ZFF6Ewa5aAtTESllr6PGsjPMZGhhK08%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldH

In [ ]:
#IMPORTANT! Do not run for inference. use for training only!
#This step is required as network will fine-tune from pre-trained models only if they are in the the models folder from superresolution
#They also need to be renamed to 1_G and 1_E as the model understands it like a checkpoint saved from previous exec.

# source and destination paths
source_path = "/content/KAIR/model_zoo/003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x4_GAN-with-dict-keys-params-and-params_ema.pth"
destination_dir = "/content/KAIR/superresolution/swinir_sr_realworld_x4_gan/models/"

# Create destination directory if not exist
os.makedirs(destination_dir, exist_ok=True)

# Copy and rename to 1_G.pth
destination_g = os.path.join(destination_dir, "1_G.pth")
shutil.copy2(source_path, destination_g)
print(f"Copied to: {destination_g}")

# Copy and rename to 1_E.pth
destination_e = os.path.join(destination_dir, "1_E.pth")
shutil.copy2(source_path, destination_e)
print(f"Copied to: {destination_e}")

# Verify the files were created
if os.path.exists(destination_g) and os.path.exists(destination_e):
    print("Both files created successfully.")
    print(f"1_G.pth size: {os.path.getsize(destination_g)} bytes")
    print(f"1_E.pth size: {os.path.getsize(destination_e)} bytes")
else:
    print("Files were not created properly.")

Copied to: /content/KAIR/superresolution/swinir_sr_realworld_x4_gan/models/1_G.pth
Copied to: /content/KAIR/superresolution/swinir_sr_realworld_x4_gan/models/1_E.pth
Both files created successfully.
1_G.pth size: 134262205 bytes
1_E.pth size: 134262205 bytes


# 3 Upload dataset from Drive

In [ ]:
dataset_zip_path = '/content/drive/MyDrive/Final_project/2000dataset.zip'
!unzip -q $dataset_zip_path -d $unzip_path

print(f"Dataset extracted to: {unzip_path}")


Dataset extracted to: /content


# 4 Fine tune swinIR GAN model

In [ ]:
# Define the JSON content
json_content = {
    "task": "swinir_sr_realworld_x4_gan",
    "model": "gan",
    "gpu_ids": [0],
    "dist": False,
    "scale": 4,
    "n_channels": 3,
    "path": {
        "root": "superresolution",
        "pretrained_netG": "/content/KAIR/superresolution/swinir_sr_realworld_x4_gan/models/1_G.pth",
        #"pretrained_netD": null,
        "pretrained_netE": "/content/KAIR/superresolution/swinir_sr_realworld_x4_gan/models/1_E.pth"
    },
    "datasets": {
        "train": {
            "name": "train_dataset",
            "dataset_type": "sr",
            "dataroot_H": "/content/dataset/train/hr",
            "dataroot_L": "/content/dataset/train/lr",
            "H_size": 256,
            "shuffle_prob": 0.1,
            "lq_patchsize": 64,
            "use_sharp": False,
            "dataloader_shuffle": True,
            "dataloader_num_workers": 2,
            "dataloader_batch_size": 2
        },
        "test": {
            "name": "test_dataset",
            "dataset_type": "sr",
            "dataroot_H": "/content/dataset/val/hr",
            "dataroot_L": "/content/dataset/val/lr"
        }
    },
    "netG": {
        "net_type": "swinir",
        "upscale": 4,
        "in_chans": 3,
        "img_size": 64,
        "window_size": 8,
        "img_range": 1.0,
        "depths": [6, 6, 6, 6, 6, 6,],
        "embed_dim": 180,
        "num_heads": [6, 6, 6, 6, 6, 6],
        "mlp_ratio": 2,
        "upsampler": "nearest+conv",
        "resi_connection": "1conv",
        "init_type": "default"
    },
    "netD": {
      "net_type": "discriminator_unet",
      "in_nc": 3,
      "base_nc": 64,
      "n_layers": 3,
      "norm_type": "spectral",
      "init_type": "orthogonal",
      "init_bn_type": "uniform",
      "init_gain": 0.2,
  }
    , "train": {
    "G_lossfn_type": "l1"
    , "G_lossfn_weight": 1

    , "F_lossfn_type": "l1"
    , "F_lossfn_weight": 1
    , "F_feature_layer": [2,7,16,25,34]
    , "F_weights": [0.1,0.1,1.0,1.0,1.0]
    , "F_use_input_norm": True
    , "F_use_range_norm": False

    , "gan_type": "gan"
    , "D_lossfn_weight": 0.1

    , "E_decay": 0.999

    , "D_init_iters": 0

    , "G_optimizer_type": "adam"
    , "G_optimizer_lr": 1e-5
    , "G_optimizer_wd": 0

    , "D_optimizer_type": "adam"
    , "D_optimizer_lr": 1e-4
    , "D_optimizer_wd": 0

    , "G_scheduler_type": "MultiStepLR"
    , "G_scheduler_milestones": [3800]
    , "G_scheduler_gamma": 0.5
    , "G_optimizer_reuse": True

    , "D_scheduler_type": "MultiStepLR"
    , "D_scheduler_milestones": [3800]
    , "D_scheduler_gamma": 0.5
    , "D_optimizer_reuse": False

    , "G_param_strict": True
    , "D_param_strict": True
    , "E_param_strict": True

    , "checkpoint_test": 100
    , "checkpoint_save": 500
    , "checkpoint_print": 100
    , "manual_seed": 134
  }
}


# Ensure directory exists
os.makedirs('/content/KAIR/options/swinir', exist_ok=True)

# Write to new file
new_json_path = '/content/KAIR/options/swinir/train_swinir_sr_realworld_x4_gan_satellite.json'
with open(new_json_path, 'w') as f:
    json.dump(json_content, f, indent=2)

print(f"New JSON created at: {new_json_path}")

New JSON created at: /content/KAIR/options/swinir/train_swinir_sr_realworld_x4_gan_satellite.json


In [ ]:
%cd /content/KAIR
!python main_train_psnr.py --opt options/swinir/train_swinir_sr_realworld_x4_gan_satellite.json --wandb_project SwinIR --wandb_name SwinIR_finetune

/content/KAIR
export CUDA_VISIBLE_DEVICES=0
number of GPUs is: 1
wandb: WARNING `start_method` is deprecated and will be removed in a future version of wandb. This setting is currently non-functional and safely ignored.
wandb: Currently logged in as: dsha43925 (dsha43925-middlesex-university-mauritius) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: ⢿ Waiting for wandb.init()...
wandb: ⣻ creating run (0.2s)
wandb: ⣽ creating run (0.2s)
wandb: Tracking run with wandb version 0.21.3
wandb: Run data is saved locally in /content/KAIR/wandb/run-20250914_185648-h98lkbp4
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run SwinIR_finetune
wandb: ⭐️ View project at https://wandb.ai/dsha43925-middlesex-university-mauritius/SwinIR
wandb: 🚀 View run at https://wandb.ai/dsha43925-middlesex-university-mauritius/SwinIR/runs/h98lkbp4
LogHandlers setup!
25-09-14 18:56:49.462 :   task: swinir_sr_realworld_x4_gan
  model: gan
  gpu_ids: [0]
  dist: False
  scal

# 5 Inference with latest generator obtained

## 5.1 unzip file with test images

In [ ]:
#unzip dataset of images to test
!unzip -q "/content/drive/MyDrive/Final_project/LRInferenceImages.zip" -d "/content"

replace /content/LRInferenceImages/AirbusDhirubhai_AmbaniSolarPlant.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


## 5.2 Inference

### for simple image test

In [ ]:
#modify with latest generator obtained
latest_model = "/content/KAIR/superresolution/swinir_sr_realworld_x4_gan/models/2000_G.pth"
#latest_model = "/content/drive/MyDrive/Final_project/SwinIR/2000_G.pth"
image_folder_path = "/content/LRInferenceImages"

In [ ]:
#/content/KAIR/SwinIR/results/swinir_real_sr_x4_large
!python main_test_swinir.py --task real_sr --scale 4 --model_path $latest_model --folder_lq $image_folder_path --tile 256

/usr/local/lib/python3.12/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
loading model from /content/KAIR/superresolution/swinir_sr_realworld_x4_gan/models/2000_G.pth
/usr/local/lib/python3.12/dist-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4322.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Testing 0 AirbusDhirubhai_AmbaniSolarPlant
Testing 1 AirbusRotterdamPort 
Testing 2 AirbusSantiagoAirport
Testing 3 MaxarDRC            
Testing 4 MaxarIndonesia      
Testing 5 MaxarNepalPostFlood 
Testing 6 MaxarYellowStone    
Testing 7 TurkeyEarthquake    


### For test dataset

In [ ]:
#Upload the zip file containing low resolution images for inference.
code_zip_path = '/content/drive/MyDrive/Final_project/testImages.zip'
unzip_path = '/content'
!unzip -q $code_zip_path -d $unzip_path
!ls -l $unzip_path

total 24
drwxrwxrwx  4 root root 4096 Aug 24 08:09 dataset
drwx------  5 root root 4096 Sep 14 17:09 drive
drwxrwxrwx 11 root root 4096 Sep 14 17:14 KAIR
drwxrwxrwx  2 root root 4096 Sep  1 20:57 LRInferenceImages
drwxr-xr-x  1 root root 4096 Sep  9 13:46 sample_data
drwxrwxrwx  7 root root 4096 Sep 11 19:39 testImages


In [ ]:
test_image_path = "/content/testImages"
folders = os.listdir(test_image_path)
model_path = "/content/KAIR/superresolution/swinir_sr_realworld_x4_gan/models/2000_G.pth"
#model_path = "/content/KAIR/model_zoo/003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x4_GAN-with-dict-keys-params-and-params_ema.pth"
for folder in folders:
    image_path = os.path.join(test_image_path, folder)
    if os.path.isdir(image_path):
        print(f"Processing {folder}")
        output_folder = f"/content/KAIR/results/swinir_real_sr_x4/{folder}"
        os.makedirs(output_folder, exist_ok=True)

        !python main_test_swinir.py --task real_sr --scale 4 --model_path "{model_path}" --folder_lq "{image_path}" --tile 256


Processing buildings
/usr/local/lib/python3.12/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
loading model from /content/KAIR/superresolution/swinir_sr_realworld_x4_gan/models/2000_G.pth
/usr/local/lib/python3.12/dist-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4322.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Testing 0 buildings2_patch_0  
Testing 1 buildings2_patch_1  
Testing 2 buildings2_patch_10 
Testing 3 buildings2_patch_11 
Testing 4 buildings2_patch_12 
Testing 5 buildings2_patch_13 
Testing 6 buildings2_patch_14 
Testing 7 buildings2_patch_15 
Testing 8 buildings2_patch_16 
Testing 9 bui

In [ ]:
# Base results folder
base_folder = "/content/KAIR/results/swinir_real_sr_x4"

# List of category folders
categories = ["buildings", "desert", "snowregion", "vegetation", "water"]

# Iterate over all files in the base folder
for fname in os.listdir(base_folder):
    fpath = os.path.join(base_folder, fname)
    if os.path.isfile(fpath):
        # Check which category the filename contains
        for cat in categories:
            if cat in fname.lower():
                dest_folder = os.path.join(base_folder, cat)
                shutil.move(fpath, os.path.join(dest_folder, fname))
                break  # once moved, stop checking other categories

# Zip the entire results folder
zip_path = "/content/swinir/FineTuned"
shutil.make_archive(zip_path, 'zip', base_folder)

# Download the zip
files.download(f"{zip_path}.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>